# Simulated OPX

In [7]:
import time
from functools import wraps


from pprint import pprint
import numpy as np
import qcodes as qc

from labcore.measurement import *

from configuration import QMConfig
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager

from plottr.data import datadict_storage as dds, datadict as dd

# global module variable for the config file
global_config = None


# variable used for development
test_name_counter = 0


In [2]:
class dummy_opx:
    def __init__(self, shape, sleep, num_lines):
        self.shape = shape 
        self.sleep = sleep
        self.num_lines = num_lines
        
        self.processing = True
        self.counter = 0
        
    def is_processing(self):
        return self.processing
    
    
    def get_data(self, batch):
        if self.processing:
            ret = []
            for i in range(len(self.shape)):
                if i == 0:
                    ret.append(np.arange(batch))
                else:
                    ret.append(np.random.randint(0,10, (batch, self.shape[i])))
            self.counter += batch
            if self.counter >= self.num_lines:
                self.processing = False
            time.sleep(self.sleep)
            return tuple(ret)


In [3]:
def start_experiment(): 
    print('running the experiment in the OPX (not implemented)')

def conditional_generator(control):
    """
    Generator that returns a list of increasing integers as long as the argument control has a boolean
    variable called active that is True. The generator stops when the variable becomes False. 
    """
    counter_internal = -1

    # In the while goes whatever condition needs to be done to stop measuring
    while control.is_processing():
        counter_internal += 1
        yield counter_internal
        
        
def create_sweep(opx_program, batchsize, num_lines):
    # instantiating necessary objects
    @recording(
        independent('repetition', type='array'),
        dependent('variable', type='array'),
        dependent('independent', depends_on=['repetition','variable'], type='array'))
    def gather_data():

        data = 0
        data = result_handle.get_data(batchsize)
        return tuple(data)
    
    # This would instantiate all of the QM stuff and return the reulst handle
    
    result_handle = opx_program((1,10,10), 1, num_lines)
    sweep_param = sweep_parameter('ignore_param', conditional_generator(result_handle), record_as(gather_data))
    sweep = Sweep(once(start_experiment) + sweep_param)
    return(sweep)

def create_structure(sweep):
    data_specs = sweep.get_data_specs()
    data_dict = dd.DataDict()
    for spec in data_specs:
        print(spec)
        depends_on = spec.depends_on
        unit = spec.unit
        name = spec.name
        if name != 'ignore_param':
            if depends_on is None:
                if unit is None:
                    data_dict[name] = dict()
                else:
                    data_dict[name] = dict(unit = unit)
            else:
                if unit is None:
                    data_dict[name] = dict(axes = depends_on[1:])
                else:
                    data_dict[name] = dict(axes = depends_on[1:], unit = unit)

    print(data_dict)
    data_dict.validate()
                
    return data_dict

def check_none(argument):
    for arg in argument.keys():
        if argument[arg] is not None:
            return False
    return True

def run_sweep(sweep, data_dir, name, prt=True):
    data_dict = create_structure(sweep)
    with dds.DDH5Writer(data_dir, data_dict, name=name) as writer:
        for line in sweep:
            if not check_none(line):
                line.pop('ignore_param')
                if prt:
                    print(line)
                writer.add_data(**line)
    print('The measurement has finished and all of the data has been saved.')

In [4]:
#program = dummy_opx((1,1,1,5), 1)
DATADIR = './data/'
sweep = create_sweep(dummy_opx, 5, 50)

In [5]:
run_sweep(sweep, DATADIR, 'test_3')

DataSpec(name='ignore_param', depends_on=None, type=<DataType.scalar: 'scalar'>, unit='')
DataSpec(name='repetition', depends_on=None, type=<DataType.array: 'array'>, unit='')
DataSpec(name='variable', depends_on=['ignore_param'], type=<DataType.array: 'array'>, unit='')
DataSpec(name='independent', depends_on=['ignore_param', 'repetition', 'variable'], type=<DataType.array: 'array'>, unit='')
{'repetition': {'unit': ''}, 'variable': {'axes': [], 'unit': ''}, 'independent': {'axes': ['repetition', 'variable'], 'unit': ''}}
Data location:  ./data/2021-07-26\2021-07-26_0023_test_3\2021-07-26_0023_test_3.ddh5
running the experiment in the OPX (not implemented)
{'repetition': array([0, 1, 2, 3, 4]), 'variable': array([[1, 0, 3, 3, 4, 4, 2, 0, 2, 7],
       [6, 7, 2, 7, 3, 3, 2, 6, 6, 8],
       [3, 7, 1, 7, 3, 2, 1, 5, 3, 6],
       [9, 8, 2, 8, 7, 4, 6, 3, 0, 7],
       [4, 9, 7, 7, 5, 2, 8, 4, 4, 4]]), 'independent': array([[0, 7, 4, 8, 7, 0, 5, 8, 0, 2],
       [1, 1, 6, 0, 9, 8, 6, 4, 

# Real OPX test

In [9]:
class ResHandleContainer:
    def __init__(self, res_handle=None):
        self.res_handle = res_handle
        self.counter = 0
        self.active = True
        
    def get_res_handle(self):
        return self.res_handle
    
    def set_res_handle(self, res_handle):
        self.res_handle = res_handle  

def record_qua_output(*data_specs):
    container_inside = ResHandleContainer()
    
    def qua_function(func):
        def nested_function(**kwarg):
            qmachine_mgr = QuantumMachinesManager()
            qmachine = qmachine_mgr.open_qm(global_config)
            job = qmachine.execute(func(**kwarg))
            result_handle = job.result_handles
            container_inside.set_res_handle(result_handle)
            print(f'the instance of container that I am getting is: {container_inside}')
        return nested_function, container_inside, *data_specs
    return qua_function

In [10]:
def create_sweep_opx(opx_program, config, batchsize):
    
    def control_progress(res_handle_in, counter_in, batchsize_in):
        ready = False
        first = True
        print('starting control_progress')
        while not ready:
            for name, handle in res_handle_in:
                    current_in = handle.count_so_far()
                    
                    if res_handle_in.is_processing():
                        if current_in - counter_in >= batchsize_in:
                            ready = True
                        else:
                            ready = False
                    else:
                        ready = True
                        container.active = False
                        print('I have turned container False')
                        
        return ready
    
    @recording(*list(opx_program[2:]))
    def gather_data():
        print('entering gathering data')

        result_handle = container.get_res_handle()
        
        # checking if the parameters passed match the parameters in the opx
        data_specs_names = [x.name for x in opx_program[2:]]
        variable_counter = 0
        for name, handle in result_handle:
            print(name)
            if name not in data_specs_names:
                raise ValueError(f'{name} is not a recorded variable')
            else:
                variable_counter += 1
        if variable_counter != len(data_specs_names):
            raise ValueError(f'Number of recorded variables ({variable_counter}) does not match number of variables gathered from the OPX ({len(data_specs_names)})')
        
        
        while container.active:

            first = True
            data = {}
            counter = container.counter
            first = True
            current = 0
            
            if result_handle == None:
                yield None
            
            record = control_progress(result_handle, counter, batchsize)  
            for name, handle in result_handle:

                if first:
                    current = handle.count_so_far()
                    print(f'getting new current: {current}, my old counter is: {counter}')
                    first = False
                    if current == counter:
                        yield None
                    
                    
                handle.wait_for_values(current)
                data_temp = np.array(handle.fetch(slice(counter, current)))     
                if name[0:4] == 'raw_':
                    holding_converting = []
                    for i in data_temp:
                        i_holder = []
                        for j in i:
                            converted = j.astype(float)
                            i_holder.append(converted)
                        holding_converting.append(i_holder)
                    if len(holding_converting) == 1:
                        converted_data_temp = [np.squeeze(holding_converting)]
                    else:
                        converted_data_temp = np.squeeze(holding_converting)
                else:
                    converted_data_temp = data_temp.astype(float)
                data[name] = converted_data_temp
            container.counter = current
            yield data
            
    my_experiment = opx_program[0]
    print(f'my experiment is: {my_experiment}')
    container = opx_program[1]

    sweep = Sweep(once(my_experiment)) + Sweep(gather_data())
#     sweep = Sweep(once(my_qua_experiment)) + Sweep(gather_data)
    return(sweep)

def create_structure(sweep):
    data_specs = sweep.get_data_specs()
    print(f'the data_specs are: {data_specs}')
    data_dict = dd.DataDict()
    for spec in data_specs:
        print(spec)
        depends_on = spec.depends_on
        unit = spec.unit
        name = spec.name
        if depends_on is None:
            if unit is None:
                data_dict[name] = dict()
            else:
                data_dict[name] = dict(unit = unit)
        else:
            if unit is None:
                data_dict[name] = dict(axes = depends_on)
            else:
                data_dict[name] = dict(axes = depends_on, unit = unit)
                    
    data_dict.validate()
    print(data_dict)
    return data_dict

def check_none(argument):
    for arg in argument.keys():
        if argument[arg] is not None:
            return False
    return True

def run_sweep(sweep, data_dir, name, prt=False):
    data_dict = create_structure(sweep)
    with dds.DDH5Writer(data_dir, data_dict, name=name) as writer:
        for line in sweep:
            if not check_none(line):
                if prt:
                    print(line)
                try:
                    writer.add_data(**line)
                except ValueError as e:
                    print(line)
                    raise e
                    
    print('The measurement has finished and all of the data has been saved.')

In [11]:

# You should be able to pass arguments to my_qua_experiment(n_reps=1_000)

@record_qua_output(
        independent('repetition', type='array'),
        dependent('V', depends_on=['repetition'], type='array'),
        dependent('tracker', depends_on=['repetition'], type='array'))
#         dependent('raw', depends_on=['repetition'], type='array'))
    #qua_raw_dependent('raw', depends_on=['repetition'], type='array'))
def my_qua_experiment(n_reps=1000):
    with program() as qua_measurement:
        raw_stream = declare_stream(adc_trace=True)
        v_stream = declare_stream()
        tracker_stream = declare_stream()
        i_stream = declare_stream()

        i = declare(int)
        v = declare(fixed)
        tracker = declare(int, value=0)

        with for_(i, 0, i<n_reps, i+1):
            save(i, i_stream)
            
            measure('box', 'readout', raw_stream, ("box_sin",v))
            save(v, v_stream)

            assign(tracker, tracker+2)
            save(tracker, tracker_stream)

            play('box', "readout")
            wait(1000000)
#             wait(1000000)

        with stream_processing():
            i_stream.save_all('repetition')
            v_stream.save_all('V')
            tracker_stream.save_all('tracker')
#             raw_stream.input1().timestamps().save_all('raw_stream')

    return qua_measurement

In [12]:
DATADIR = './data/'
config = QMConfig()
global_config = config.config()
sweep = create_sweep_opx(my_qua_experiment, config.config(), 5)

sweep.set_action_opts(
    nested_function=dict(n_reps=5),
)

my experiment is: <function record_qua_output.<locals>.qua_function.<locals>.nested_function at 0x000001F774D2BB80>


In [13]:
test_name_counter += 1
run_sweep(sweep, DATADIR, f'OPX test #{test_name_counter}', prt=False)

the data_specs are: (DataSpec(name='repetition', depends_on=None, type=<DataType.array: 'array'>, unit=''), DataSpec(name='V', depends_on=['repetition'], type=<DataType.array: 'array'>, unit=''), DataSpec(name='tracker', depends_on=['repetition'], type=<DataType.array: 'array'>, unit=''))
DataSpec(name='repetition', depends_on=None, type=<DataType.array: 'array'>, unit='')
DataSpec(name='V', depends_on=['repetition'], type=<DataType.array: 'array'>, unit='')
DataSpec(name='tracker', depends_on=['repetition'], type=<DataType.array: 'array'>, unit='')
{'repetition': {'unit': '', 'axes': [], 'label': '', 'values': array([], dtype=float64)}, 'V': {'axes': ['repetition'], 'unit': '', 'label': '', 'values': array([], dtype=float64)}, 'tracker': {'axes': ['repetition'], 'unit': '', 'label': '', 'values': array([], dtype=float64)}}
Data location:  ./data/2021-07-28\2021-07-28_0002_OPX test #2\2021-07-28_0002_OPX test #2.ddh5
2021-07-28 17:21:38,142 - qm - INFO - Performing health check
2021-07

None


# Structured with base class decorator

In [46]:
class BackgroundRecordingBase:
    """
    Base class decorator used to record asynchronous data from instrument. 
    Each instrument should have its own custom setup_wrapper (see setup_wrapper docstring for more info), and a custom collector 
    """
    def __init__(self, *specs):
        
        self.specs = specs  
        self.communicator = {}

        
    def __call__(self, fun):
        def sweep(**collector_kwargs):
            setup_sweep = once(self.setup_wrapper(fun))
            gather_sweep = Sweep(record_as(self.collector(**collector_kwargs), *self.specs))
            return setup_sweep + gather_sweep
        return sweep
    
    
        
    def setup_wrapper(self, fun):
        """
        Wraps the setup function. setup_wrapper should consist of a function wrapped by the decorator @wraps and takes fun as an arugment.
        In this case the wrapped function is setup. 
        Setup should accpet the *args and **kwargs of fun. It should also place any returns from fun in the communicator.
        setup_wrapper needs to return the wraped function (setup)
        """
        @wraps(fun)
        def setup(*args, **kwargs):
            self.communicator['setup_return'] = fun(*args, **kwargs)
            return None
        
        return setup
    
    def collector(self):
        """
        Data gathering function. This function should be a generator that collects data from the instrument.
        All the logic of asynchronous data collection should be here. It should yield data as long as it is 
        available and the generator should finish once its experiment is done.
        """
        yield None
    
    
def create_background_sweep(decorated_setup_function, **collector_kwargs):
    sweep = decorated_setup_function(**collector_kwargs)
    return sweep


In [47]:
class RecordOPX(BackgroundRecordingBase):
    def setup_wrapper(self, fun):
        @wraps(fun)
        def setup(*args, **kwargs):
            qmachine_mgr = QuantumMachinesManager()
            qmachine = qmachine_mgr.open_qm(global_config)
            job = qmachine.execute(fun(*args, **kwargs))
            result_handles = job.result_handles
            self.communicator['result_handles'] = result_handles
            self.communicator['active'] = True
            self.communicator['counter'] = 0
            
        return setup
    
    
    def _control_progress(self, batchsize):
        ready = False
        first = True
        res_handle = self.communicator['result_handles']
        counter = self.communicator['counter']
        print('starting control_progress')
        while not ready:
            for name, handle in res_handle:
                    current = handle.count_so_far()
                    
                    if res_handle.is_processing():
                        if current - counter >= batchsize:
                            ready = True
                        else:
                            ready = False
                    else:
                        ready = True
                        self.communicator['active'] = False
                        print('I have turned container False')
                        
        return ready
    

    def collector(self, batchsize):
        print('entering gathering data')

        result_handle = self.communicator['result_handles']
        
        # checking if the parameters passed match the parameters in the opx
        data_specs_names = [x.name for x in self.specs]
        variable_counter = 0
        for name, handle in result_handle:
            print(name)
            if name not in data_specs_names:
                raise ValueError(f'{name} is not a recorded variable')
            else:
                variable_counter += 1
        if variable_counter != len(data_specs_names):
            raise ValueError(f'Number of recorded variables ({variable_counter}) \
                             does not match number of variables gathered from the OPX ({len(data_specs_names)})')
        
        
        while self.communicator['active']:

            first = True
            data = {}
            counter = self.communicator['counter']
            first = True
            current = 0
            
            if result_handle == None:
                yield None
            
            record = self._control_progress(batchsize)  
            for name, handle in result_handle:

                if first:
                    current = handle.count_so_far()
                    print(f'getting new current: {current}, my old counter is: {counter}')
                    first = False
                    if current == counter:
                        yield None
                    
                    
                handle.wait_for_values(current)
                data_temp = np.array(handle.fetch(slice(counter, current)))     
                if name[0:4] == 'raw_':
                    holding_converting = []
                    for i in data_temp:
                        i_holder = []
                        for j in i:
                            converted = j.astype(float)
                            i_holder.append(converted)
                        holding_converting.append(i_holder)
                    if len(holding_converting) == 1:
                        converted_data_temp = [np.squeeze(holding_converting)]
                    else:
                        converted_data_temp = np.squeeze(holding_converting)
                else:
                    converted_data_temp = data_temp.astype(float)
                data[name] = converted_data_temp
            self.communicator['counter'] = current
            yield data
            

In [48]:
@RecordOPX(
        independent('repetition', type='array'),
        dependent('V', depends_on=['repetition'], type='array'),
        dependent('tracker', depends_on=['repetition'], type='array'))
#         dependent('raw', depends_on=['repetition'], type='array'))
    #qua_raw_dependent('raw', depends_on=['repetition'], type='array'))
def my_qua_experiment_standard(n_reps=1000):
    with program() as qua_measurement:
        raw_stream = declare_stream(adc_trace=True)
        v_stream = declare_stream()
        tracker_stream = declare_stream()
        i_stream = declare_stream()

        i = declare(int)
        v = declare(fixed)
        tracker = declare(int, value=0)

        with for_(i, 0, i<n_reps, i+1):
            save(i, i_stream)
            
            measure('box', 'readout', raw_stream, ("box_sin",v))
            save(v, v_stream)

            assign(tracker, tracker+2)
            save(tracker, tracker_stream)

            play('box', "readout")
            wait(1000000)
#             wait(1000000)

        with stream_processing():
            i_stream.save_all('repetition')
            v_stream.save_all('V')
            tracker_stream.save_all('tracker')
#             raw_stream.input1().timestamps().save_all('raw_stream')

    return qua_measurement

In [49]:
DATADIR = './data/'
config = QMConfig()
global_config = config.config()

sweep = create_background_sweep(my_qua_experiment_standard, batchsize=5)


In [50]:
run_sweep(sweep, DATADIR, f'OPX test standard #{test_name_counter}', prt=True)

the data_specs are: (DataSpec(name='repetition', depends_on=None, type=<DataType.array: 'array'>, unit=''), DataSpec(name='V', depends_on=['repetition'], type=<DataType.array: 'array'>, unit=''), DataSpec(name='tracker', depends_on=['repetition'], type=<DataType.array: 'array'>, unit=''))
DataSpec(name='repetition', depends_on=None, type=<DataType.array: 'array'>, unit='')
DataSpec(name='V', depends_on=['repetition'], type=<DataType.array: 'array'>, unit='')
DataSpec(name='tracker', depends_on=['repetition'], type=<DataType.array: 'array'>, unit='')
{'repetition': {'unit': '', 'axes': [], 'label': '', 'values': array([], dtype=float64)}, 'V': {'axes': ['repetition'], 'unit': '', 'label': '', 'values': array([], dtype=float64)}, 'tracker': {'axes': ['repetition'], 'unit': '', 'label': '', 'values': array([], dtype=float64)}}
Data location:  ./data/2021-07-28\2021-07-28_0007_OPX test standard #2\2021-07-28_0007_OPX test standard #2.ddh5
2021-07-28 18:00:38,816 - qm - INFO - Performing he

In [13]:
config = QMConfig()
qmachine_mgr = QuantumMachinesManager()
qmachine = qmachine_mgr.open_qm(config.config())
job = qmachine.execute(my_qua_experiment())
res_handle = job.result_handles
res_handle.wait_for_all_values()
data=[]
print(type(res_handle))
for name, handle in res_handle:
    handle.wait_for_values(0)
    
    print(f'saving {name}')
    print(f"the data that comes back has a type of: {type(handle.fetch_all()['value'])}")
    data.append(np.array(handle.fetch_all()['value']))

print(data)

2021-07-27 16:40:10,426 - qm - INFO - Performing health check
2021-07-27 16:40:10,441 - qm - INFO - Health check passed
2021-07-27 16:40:10,741 - qm - INFO - Flags: 
2021-07-27 16:40:10,741 - qm - INFO - Executing high level program
<class 'qm._results.JobResults'>
saving repetition
the data that comes back has a type of: <class 'numpy.ndarray'>
saving V
the data that comes back has a type of: <class 'numpy.ndarray'>
saving tracker
the data that comes back has a type of: <class 'numpy.ndarray'>
[array([   0,    1,    2, ..., 9997, 9998, 9999], dtype=int64), array([-0.19584275, -0.19584035, -0.19582246, ..., -0.19582816,
       -0.19582617, -0.19583536]), array([    3,     5,     7, ..., 19997, 19999, 20001], dtype=int64)]


In [12]:
def gen():
    x = 0
    while x < 5:
        x = np.random.randint(low=0, high=6)
        yield x

In [15]:
sweep = sweep_parameter('x', gen(), record_as(lambda: 5, 'a'))
for rep in sweep:
    print(rep)

{'x': 3, 'a': 5}
{'x': 0, 'a': 5}
{'x': 3, 'a': 5}
{'x': 3, 'a': 5}
{'x': 3, 'a': 5}
{'x': 2, 'a': 5}
{'x': 5, 'a': 5}


In [16]:
sweep.get_data_specs()

(DataSpec(name='x', depends_on=None, type=<DataType.scalar: 'scalar'>, unit=''),
 DataSpec(name='a', depends_on=['x'], type=<DataType.scalar: 'scalar'>, unit=''))

In [17]:
for data in Sweep(range(5)):
    print(data)

{}
{}
{}
{}
{}


In [40]:
def repetitions(repeat):
    def inner_function(func):
        return func
    return inner_function, repeat

In [43]:
@repetitions(repeat = 1)
def print_something(text):
    print(text)

TypeError: 'tuple' object is not callable

TypeError: 'tuple' object is not callable

In [120]:
def f(x):
    def g(y):
        print(f'x is: {x} and y is: {y}')
        return y + x + 3 
    return g

nf1 = f(1)
nf2 = f(3)

print(nf1(5))
print(nf2(1))

x is: 1 and y is: 5
9
x is: 3 and y is: 1
7


In [142]:
def my_qua_decorator_sim(*data_specs):
    def nested_1(func):
        return func, *data_specs
    return nested_1

In [150]:
@my_qua_decorator_sim(
    independent('x', type='array'),
    dependent('y', depends_on=['x'], type='array'))
def calculation(mult=2, num=5):
    x = [i for i in range(num)]
    y = [i*mult for i in x]
    return np.array(x), np.array(y)


In [154]:
def sweep_creation(recorded_function):
    for i in recorded_function:
        print(i)
    @recording(recorded_function[1:])
    def collect():
        counter = 5
        while counter >= 0:
            counter -=1
            yield recorded_function[0]()
            
            
    return Sweep(collect())
        

In [171]:
sweep = sweep_creation(calculation)

<function calculation at 0x0000019B05A9CEE0>
DataSpec(name='x', depends_on=None, type=<DataType.array: 'array'>, unit='')
DataSpec(name='y', depends_on=['x'], type=<DataType.array: 'array'>, unit='')


In [174]:
data = []
for i in sweep:
    data.append(i)

TypeError: unhashable type: 'DataSpec'

[(array([0, 1, 2, 3, 4]), array([0, 2, 4, 6, 8])),
 (array([0, 1, 2, 3, 4]), array([0, 2, 4, 6, 8])),
 (array([0, 1, 2, 3, 4]), array([0, 2, 4, 6, 8])),
 (array([0, 1, 2, 3, 4]), array([0, 2, 4, 6, 8])),
 (array([0, 1, 2, 3, 4]), array([0, 2, 4, 6, 8]))]